In [15]:
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Script to set ensemsble dictionary with Latin Hypercube Sampling
# Mikayla Pascual 07/22/22
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

#import 
from MatlabFuncs import *
from model import *
from loadmodel import *
from os.path import exists
import lhsmdu
import numpy as np
import pickle

In [11]:
# gather parameters we want to vary
md = model()
### frontal forcings ###
from frontalforcingsrignot import frontalforcingsrignot
md.frontalforcings = frontalforcingsrignot()
md.frontalforcings.subglacial_discharge = 2.6 * np.ones(md.mesh.numberofvertices)
md.frontalforcings.thermalforcing = 15.7 * np.ones(md.mesh.numberofvertices)

### calving forcings ###
# We set the calving model (no moving front ... spclevelset is actually ignored)
from calvingvonmises import calvingvonmises
md.calving = calvingvonmises()
md.calving.stress_threshold_groundedice = md.calving.stress_threshold_groundedice * 0.5

In [12]:
# Set Latin Hypercube sampling parameters
k = lhsmdu.sample(3, 10) # Latin Hypercube Sampling with multi-dimensional uniformity, 
                        # 3 variables with 10 samples each
k = np.array(k)
# collect -20% to 20% range
k = k * 0.4 - 0.2

# change initial model parameters to proportional LHS output
sd_after = 2.6 * k[0]
tf_after = 15.7 * k[1]
calve_after = md.calving.stress_threshold_groundedice + md.calving.stress_threshold_groundedice*k[2]

In [13]:
# Create 10 ensembles with parameters from [0], [1], [2], ..., [9]
lhs_params_for_ens = {
    'sd_after' : sd_after,
    'tf_after' : tf_after,
    'calve_after' : calve_after
} 
lhs_params_for_ens

# ens needs to match LHS sample number
ens = list(range(0, 10))

ens_dict = {}
for i in range(len(ens)):
    trial_name = 'Trial' + str(i)
    res = dict()
    for sub in lhs_params_for_ens:
        res[sub] = lhs_params_for_ens[sub][i]
    ens_dict[trial_name] = res
ens_dict

{'Trial0': {'sd_after': 0.3456626405860415,
  'tf_after': 1.5904661636423953,
  'calve_after': 415204.55719779374},
 'Trial1': {'sd_after': 0.46024538961610745,
  'tf_after': -3.107454818875516,
  'calve_after': 431912.7748121569},
 'Trial2': {'sd_after': -0.05120852061681667,
  'tf_after': 2.095387486705782,
  'calve_after': 529431.5237710031},
 'Trial3': {'sd_after': 0.1292106121711683,
  'tf_after': -1.1715875828823243,
  'calve_after': 588236.8182829454},
 'Trial4': {'sd_after': -0.4040569702271229,
  'tf_after': 2.5517994814568987,
  'calve_after': 506977.3653308599},
 'Trial5': {'sd_after': -0.24849551558607022,
  'tf_after': -1.2623049740591141,
  'calve_after': 458590.5828849565},
 'Trial6': {'sd_after': 0.23801757753698652,
  'tf_after': 0.8304382146441261,
  'calve_after': 576612.3881557547},
 'Trial7': {'sd_after': -0.1475512249720483,
  'tf_after': -0.11939884800328923,
  'calve_after': 499300.53821333026},
 'Trial8': {'sd_after': -0.503946277611483,
  'tf_after': 0.1599143

In [16]:
# save output
with open('ens_dict.pickle', 'wb') as f:
    # Pickle the 'data' dictionary 
    pickle.dump(ens_dict, f)